In [16]:
import pandas as pd

def read_data_from_spreadsheet(filename, cs=19):
    sheet = pd.read_excel(filename)

    cs = min(cs, 19)
    
#    dependencies = sheet.iloc[:,2:21].values.tolist()
    
    dependencies = sheet.iloc[0:cs,2:2+cs].values.tolist()

    return dependencies

In [21]:
import pulp

def get_data(filename, size=19):
    
    size = min(size, 19)
    
    profitability = [85,80,70,75,35,45,50,75,70,55,60,35,30,15,20,75,80,75,45]
    profitability = profitability[0:size]
    story_points = [8,7,3,6,4,5,5,7,8,6,7,3,4,6,6,9,8,7,4]
    story_points = story_points[0:size]
    sprint_capacity = [15,15,20,20,25,25]
    
    num_user_stories = len(story_points)
    num_sprints = len(sprint_capacity)

    dependencies = read_data_from_spreadsheet(filename, size)

    return num_user_stories, num_sprints, profitability, story_points, sprint_capacity, dependencies

def solve_user_story_assignment(filename, size=19):
    num_user_stories, num_sprints, profitability, story_points, sprint_capacity, dependencies = get_data(filename, size)

    # Create a PuLP minimization problem
    model = pulp.LpProblem("User_Story_Assignment", pulp.LpMaximize)

    # Create decision variables
    x = []
    for i in range(num_user_stories):
        x.append([])
        for j in range(num_sprints):
            x[i].append(pulp.LpVariable(f"x_{i}_{j}", cat=pulp.LpBinary))

    # Set the objective function
    model += sum(profitability[i] * x[i][j] for i in range(num_user_stories) for j in range(num_sprints))

    # Set the constraints
    for j in range(num_sprints):
        model += sum(story_points[i] * x[i][j] for i in range(num_user_stories)) <= sprint_capacity[j]

    for i in range(num_user_stories):
        model += sum(x[i][j] for j in range(num_sprints)) == 1

    for i in range(num_user_stories):
        for j in range(num_user_stories):
            for k in range(num_sprints):
                if i != j and dependencies[i][j] == 1:
                    model += sum(x[j][t] for t in range(k+1)) >= sum(x[i][t] for t in range(k+1))

    # Solve the problem
    model.solve()

    # Print the solution status
    print("Status:", pulp.LpStatus[model.status])

    # Print the optimal objective value
    print("Objective Value:", pulp.value(model.objective))

    # Print the optimal assignment
    for i in range(num_user_stories):
        for j in range(num_sprints):
            if pulp.value(x[i][j]) == 1:
                print(f"User Story {i} assigned to Sprint {j}")



In [22]:
filename = "Interdependences_var09.xlsx"

for i in range(5, 20):
    print('Attempt', i)
    solve_user_story_assignment(filename, i)

Attempt 5
Status: Optimal
Objective Value: 345.0
User Story 0 assigned to Sprint 3
User Story 1 assigned to Sprint 0
User Story 2 assigned to Sprint 2
User Story 3 assigned to Sprint 0
User Story 4 assigned to Sprint 2
Attempt 6
Status: Infeasible
Objective Value: 389.99999938
User Story 1 assigned to Sprint 0
Attempt 7
Status: Infeasible
Objective Value: 439.99999937999996
User Story 1 assigned to Sprint 0
User Story 6 assigned to Sprint 2
Attempt 8
Status: Infeasible
Objective Value: 515.0
User Story 1 assigned to Sprint 0
User Story 6 assigned to Sprint 2
User Story 7 assigned to Sprint 4
Attempt 9
Status: Infeasible
Objective Value: 585.0
User Story 1 assigned to Sprint 0
Attempt 10
Status: Infeasible
Objective Value: 640.0
User Story 1 assigned to Sprint 0
Attempt 11
Status: Infeasible
Objective Value: 700.0
User Story 1 assigned to Sprint 0
User Story 10 assigned to Sprint 4
Attempt 12
Status: Infeasible
Objective Value: 734.9999999999999
Attempt 13
Status: Infeasible
Objective V